In [1]:
from keras import backend as K

C:\Users\BrandonSchabell\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# just to see the input format for input_shape
K.image_data_format()

'channels_last'

In [3]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Dense, \
    Flatten, Activation
from keras.regularizers import l2, l1
from keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np

In [4]:
training_directory = os.getcwd() + os.sep + "Images"
test_directory = os.getcwd() + os.sep + "TestImages"

train_samples = 27992
test_samples = 7000
# train_samples = 33033
# test_samples = 8254

In [5]:
# tried to have to conv layers per each pooling but will take to long to run as of now
# so far not looking like a very good model, it is not yet finished for me
# would like to have more inputs on the first dense layer but also would take too long 

model = Sequential()
model.add(Convolution2D(32, (5, 5), activation='relu', input_shape=(405,270, 3)))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Activation('relu'))

model.add(Convolution2D(32, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Activation('relu'))

model.add(Convolution2D(32, (5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(70, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(70, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics=['accuracy'])

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 401, 266, 32)      2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 133, 88, 32)       0         
_________________________________________________________________
activation_1 (Activation)    (None, 133, 88, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 129, 84, 32)       25632     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 43, 28, 32)        0         
_________________________________________________________________
activation_2 (Activation)    (None, 43, 28, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 39, 24, 32)        25632     
__________

In [7]:
# would like to use more epochs but will take to long.

datagen = ImageDataGenerator(rescale=1. / 255)
batch_size = 50

train_generator = datagen.flow_from_directory(
    training_directory,
    target_size=(405, 270),
    batch_size= batch_size,
    class_mode='categorical')

test_generator = datagen.flow_from_directory(
    test_directory,
    target_size=(405, 270),
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch = train_samples // batch_size,
    epochs = 50,
    validation_data = test_generator,
    validation_steps = test_samples // batch_size)

model.save_weights('starting_point.h5')

Found 27992 images belonging to 70 classes.
Found 7001 images belonging to 70 classes.
Epoch 1/2
559/559 [==============================] - 102s 183ms/step - loss: 4.2497 - acc: 0.0140 - val_loss: 4.2493 - val_acc: 0.0124
Epoch 2/2
559/559 [==============================] - 101s 180ms/step - loss: 4.1234 - acc: 0.0341 - val_loss: 3.5949 - val_acc: 0.1208


In [8]:
predicted = model.predict_generator(test_generator, test_samples // batch_size)
predicted = np.argmax(predicted, axis=1)

In [9]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, predicted))

print(accuracy_score(test_generator.classes, predicted))

Confusion Matrix


ValueError: Found input variables with inconsistent numbers of samples: [7001, 7000]